# Week 5 & 6 deliverables

Olivia Whitelaw 
V01001981

Hours: 10


## Imports

In [82]:
import sys
import os
import pandas as pd
import subprocess
import requests
import shutil
from pathlib import Path
import urllib.request
import gzip
import ssl

## Step 1: 

Fetch the reference genomic sequence (FASTA file) for the chromosomes containing CYP2C8, CYP2C9, and CYP2C19 genes from the hg38 (GRCh38) human genome build.

In [56]:
# ensure data directory exists
os.makedirs("data", exist_ok=True)

fasta_path = "data/chr10.fa"
gz_path = "data/chr10.fa.gz"

# Disable SSL verification temporarily 
ssl._create_default_https_context = ssl._create_unverified_context

if os.path.exists(fasta_path) and os.path.getsize(fasta_path) > 1_000_000:
    print("reference genome already exists")

else:
    url = "https://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr10.fa.gz"
    print("Downloading reference genome from:", url)
    
    # Step 2. Download the gzipped file
    urllib.request.urlretrieve(url, gz_path)
    print("✓ Download complete:", gz_path)
    
    # Step 3. Unzip it
    print("Unzipping...")
    with gzip.open(gz_path, "rb") as f_in:
        with open(fasta_path, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)
    print("Unzipped to:", fasta_path)

    # remove compressed file if it exists
    if os.path.exists("data/chr10.fa.gz"):
        os.remove("data/chr10.fa.gz")

print("done")

✓ Download complete: data/chr10.fa.gz
Unzipping...
Unzipped to: data/chr10.fa
done


## Step 2.1: 

Download sequencing data illumina and pacbio (2 types of raw DNA sequencing data). Each come with FASTQ files

In [57]:
import bz2

# URLs for sample data
illumina_path = "https://github.com/inumanag/fall25-csc-bioinf/raw/refs/heads/main/week4/data/illumina.fq.bz2"
pacbio_path = "https://github.com/inumanag/fall25-csc-bioinf/raw/refs/heads/main/week4/data/pacbio.fq.bz2"

illumina_file = "data/illumina.fq.bz2"
pacbio_file   = "data/pacbio.fq.bz2"

for url, out in [(illumina_path, illumina_file), (pacbio_path, pacbio_file)]:
    print(f"Downloading {url} ...")
    r = requests.get(url)
    with open(out, "wb") as f:
        f.write(r.content)
    print(f"Saved to {out}")


for in_path in [illumina_file, pacbio_file]:
    out_path = in_path.replace(".bz2", "")  # remove .bz2 extension
    print(f"Decompressing {in_path} → {out_path}")
    with bz2.open(in_path, "rb") as f_in, open(out_path, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
    print("Done")


os.remove("data/illumina.fq.bz2")
os.remove("data/pacbio.fq.bz2")







Saved to data/illumina.fq.bz2
Saved to data/pacbio.fq.bz2
Decompressing data/illumina.fq.bz2 → data/illumina.fq
Done
Decompressing data/pacbio.fq.bz2 → data/pacbio.fq
Done


## Step 2.2: Create index .mmi file 

Note: you will need to download the human genome for this step; however, note that you do not need the whole human genome. Just focus on the chromosome that contains those genes! The reference should basically be a single FASTA file (extension: .fa).

Minimap2 can’t align reads directly to a FASTA file (.fa) without first building an index — a compact, searchable version of the reference genome.

minimap2 -d chr10.mmi chr10.fa
This saves time becasue minimap2 doesn’t have to re-index chr10.fa twice.



In [58]:
# create index file (chr10.mmi) that minimap2 will use 
cmd = ["minimap2", "-d", "data/chr10.mmi", "data/chr10.fa"]
subprocess.run(cmd, check=True)

[M::mm_idx_gen::2.194*0.96] collected minimizers
[M::mm_idx_gen::2.797*1.40] sorted minimizers
[M::main::3.562*1.30] loaded/built the index for 1 target sequence(s)
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::3.715*1.29] distinct minimizers: 16061920 (79.91% are singletons); average occurrences: 1.563; average spacing: 5.329; total length: 133797422
[M::main] Version: 2.30-r1287
[M::main] CMD: minimap2 -d data/chr10.mmi data/chr10.fa
[M::main] Real time: 3.755 sec; CPU: 4.830 sec; Peak RSS: 0.952 GB


CompletedProcess(args=['minimap2', '-d', 'data/chr10.mmi', 'data/chr10.fa'], returncode=0)

## Step 2.3: 

Alignment: Figure out where each read belongs in the human genome. 

Steps: 

1) Use chromosome 10 from the GRCh38/hg38 human. That .fa file will be what the reads are aligned to
2) Minimap 2 takes: (reference genome .fa file, sequencing reads .fastq file)
3) pick right setting: Illunima minimap -ax sr & PackBio -ax map-pb
4) minimap2 output text alignment as .sam


In [59]:
# Align Illumina reads
illumina_cmd = "minimap2 -ax sr data/chr10.mmi data/illumina.fq > data/illumina.sam"
subprocess.run(illumina_cmd, shell=True, check=True)

# Align PacBio reads
pacbio_cmd = "minimap2 -ax map-pb data/chr10.mmi data/pacbio.fq > data/pacbio.sam"
subprocess.run(pacbio_cmd, shell=True, check=True)

for f in ["data/illumina.sam", "data/pacbio.sam"]:
    print(f"{f}: {'exists' if os.path.exists(f) else 'missing'}")


[WARNING] Indexing parameters (-k, -w or -H) overridden by parameters used in the prebuilt index.
[M::main::0.761*0.99] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.761*0.99] mid_occ = 1000
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.921*0.99] distinct minimizers: 16061920 (79.91% are singletons); average occurrences: 1.563; average spacing: 5.329; total length: 133797422
[M::worker_pipeline::13.711*2.75] mapped 309505 sequences
[M::main] Version: 2.30-r1287
[M::main] CMD: minimap2 -ax sr data/chr10.mmi data/illumina.fq
[M::main] Real time: 13.734 sec; CPU: 37.778 sec; Peak RSS: 0.856 GB
[WARNING] Indexing parameters (-k, -w or -H) overridden by parameters used in the prebuilt index.
[M::main::0.777*0.98] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.990*0.99] mid_occ = 178
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::1.146*0.99] distinct minimizers: 16061920 (79.91

data/illumina.sam: exists
data/pacbio.sam: exists


[M::worker_pipeline::2.916*2.17] mapped 3063 sequences
[M::main] Version: 2.30-r1287
[M::main] CMD: minimap2 -ax map-pb data/chr10.mmi data/pacbio.fq
[M::main] Real time: 2.937 sec; CPU: 6.345 sec; Peak RSS: 0.764 GB


## Step 2.4: Illumina Reads

Use samtools to convert them to .bam, sort them and index them to make .bai files

In [60]:
# paths
ref_idx = Path("data/chr10.mmi")
illumina_fastq = Path("data/illumina.fq")

sam_out = Path("data/illumina_align.sam")
bam_raw = Path("data/illumina_raw.bam")
bam_final = Path("data/illumina.aligned.bam")
bai_final = Path("data/illumina.aligned.bam.bai")


# Skip if already done
if bam_final.exists() and bai_final.exists():
    print(f"Alignment exists: {bam_final}")
else:
    # 1) Run minimap2 alignment
    print("→ Running minimap2 for Illumina reads...")
    align_cmd = (
        f"minimap2 -ax sr -t 4 {ref_idx} {illumina_fastq} "
        f"> {sam_out}"
    )
    subprocess.run(align_cmd, shell=True, check=True)
    print(f"   SAM saved → {sam_out}")

    # 2) Convert SAM → BAM
    print("→ Converting SAM → BAM...")
    subprocess.run(
        f"samtools view -b {sam_out} > {bam_raw}",
        shell=True, check=True
    )
    print(f"   BAM saved → {bam_raw}")

    # 3) Sort BAM
    print("→ Sorting BAM...")
    subprocess.run(
        f"samtools sort {bam_raw} -o {bam_final}",
        shell=True, check=True
    )
    print(f"   Sorted BAM → {bam_final}")

    # 4) Index BAM
    print("→ Indexing BAM...")
    subprocess.run(f"samtools index {bam_final}", shell=True, check=True)
    print(f"   Index created → {bai_final}")

    # 5) Remove temporary files
    sam_out.unlink(missing_ok=True)
    bam_raw.unlink(missing_ok=True)

    print("\nFinished Illumina alignment ")
    print(f"Output files:\n ├─ {bam_final}\n └─ {bai_final}")




→ Running minimap2 for Illumina reads...


[WARNING] Indexing parameters (-k, -w or -H) overridden by parameters used in the prebuilt index.
[M::main::0.789*0.97] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.789*0.97] mid_occ = 1000
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.986*0.98] distinct minimizers: 16061920 (79.91% are singletons); average occurrences: 1.563; average spacing: 5.329; total length: 133797422
[M::worker_pipeline::10.697*3.54] mapped 309505 sequences
[M::main] Version: 2.30-r1287
[M::main] CMD: minimap2 -ax sr -t 4 data/chr10.mmi data/illumina.fq
[M::main] Real time: 10.722 sec; CPU: 37.854 sec; Peak RSS: 0.909 GB


   SAM saved → data/illumina_align.sam
→ Converting SAM → BAM...
   BAM saved → data/illumina_raw.bam
→ Sorting BAM...
   Sorted BAM → data/illumina.aligned.bam
→ Indexing BAM...
   Index created → data/illumina.aligned.bam.bai

Finished Illumina alignment 
Output files:
 ├─ data/illumina.aligned.bam
 └─ data/illumina.aligned.bam.bai


## Step 2.5: PacBio Reads

Use samtools to convert them to .bam, sort them and index them to make .bai files

In [61]:
# Paths
ref_idx = Path("data/chr10.mmi")
pacbio_fastq = Path("data/pacbio.fq")

sam_out = Path("data/pacbio_align.sam")
bam_raw = Path("data/pacbio_raw.bam")
bam_final = Path("data/pacbio.aligned.bam")
bai_final = Path("data/pacbio.aligned.bam.bai")

print("=== PacBio Alignment Pipeline ===")

if bam_final.exists() and bai_final.exists():
    print(f"✔ Alignment already exists: {bam_final}")
else:
    # 1️⃣ Run minimap2 with PacBio preset
    print("→ Running minimap2 for PacBio reads...")
    align_cmd = (
        f"minimap2 -ax map-pb -t 4 {ref_idx} {pacbio_fastq} > {sam_out}"
    )
    subprocess.run(align_cmd, shell=True, check=True)
    print(f"   SAM saved → {sam_out}")

    # 2️⃣ Convert SAM → BAM
    print("→ Converting SAM → BAM...")
    subprocess.run(f"samtools view -b {sam_out} > {bam_raw}", shell=True, check=True)
    print(f"   BAM saved → {bam_raw}")

    # 3️⃣ Sort BAM
    print("→ Sorting BAM...")
    subprocess.run(f"samtools sort {bam_raw} -o {bam_final}", shell=True, check=True)
    print(f"   Sorted BAM → {bam_final}")

    # 4️⃣ Index BAM
    print("→ Indexing BAM...")
    subprocess.run(f"samtools index {bam_final}", shell=True, check=True)
    print(f"   Index created → {bai_final}")

    # 5️⃣ Cleanup
    print("🧹 Cleaning up temporary files...")
    sam_out.unlink(missing_ok=True)
    bam_raw.unlink(missing_ok=True)

    print("\nFinished PacBio alignment")
    print(f"Output files:\n ├─ {bam_final}\n └─ {bai_final}")




=== PacBio Alignment Pipeline ===
→ Running minimap2 for PacBio reads...


[WARNING] Indexing parameters (-k, -w or -H) overridden by parameters used in the prebuilt index.
[M::main::0.845*0.93] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::1.224*0.92] mid_occ = 178
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::1.371*0.93] distinct minimizers: 16061920 (79.91% are singletons); average occurrences: 1.563; average spacing: 5.329; total length: 133797422
[M::worker_pipeline::2.692*2.37] mapped 3063 sequences
[M::main] Version: 2.30-r1287
[M::main] CMD: minimap2 -ax map-pb -t 4 data/chr10.mmi data/pacbio.fq
[M::main] Real time: 2.720 sec; CPU: 6.419 sec; Peak RSS: 0.724 GB


   SAM saved → data/pacbio_align.sam
→ Converting SAM → BAM...
   BAM saved → data/pacbio_raw.bam
→ Sorting BAM...
   Sorted BAM → data/pacbio.aligned.bam
→ Indexing BAM...
   Index created → data/pacbio.aligned.bam.bai
🧹 Cleaning up temporary files...

Finished PacBio alignment
Output files:
 ├─ data/pacbio.aligned.bam
 └─ data/pacbio.aligned.bam.bai


## Step 3: 

Sample reads: have aligned reads for .bam and .bai files

Use bcftools to find places in the genome where sample reads differ from the reference chr10.fa (SNPs and indels)

Needed: 

Reference genome: data/chr10.fa

Aligned reads: data/illumina.sorted.bam and data/pacbio.sorted.bam

Output: data/illumina.vcf and data/pacbio.vcf

## Step 3: Variant Calling

In [62]:
# Illumina
print("→ Calling variants for Illumina sample...")
subprocess.run(
    "bcftools mpileup -Ou -f data/chr10.fa data/illumina.aligned.bam | "
    "bcftools call -mv -Ov -o data/illumina.vcf",
    shell=True, check=True
)
print("✓ Created data/illumina.vcf")

# PacBio
print("→ Calling variants for PacBio sample...")
subprocess.run(
    "bcftools mpileup -Ou -f data/chr10.fa data/pacbio.aligned.bam | "
    "bcftools call -mv -Ov -o data/pacbio.vcf",
    shell=True, check=True
)
print("✓ Created data/pacbio.vcf")

print("\n✅ Variant calling complete! VCF files created successfully.")



→ Calling variants for Illumina sample...


Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250


✓ Created data/illumina.vcf
→ Calling variants for PacBio sample...


[mpileup] 1 samples in 1 input files
Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
[mpileup] maximum number of reads per input file set to -d 250


✓ Created data/pacbio.vcf

✅ Variant calling complete! VCF files created successfully.


## Step 4: Phase the variant VCFs 



In [63]:
# define file paths 
illumina_bam = Path("data/illumina.aligned.bam")
illumina_vcf = Path("data/illumina.vcf")
illumina_frag = Path("data/illumina.fragments")
illumina_hapcut = Path("data/illumina.hapcut")

pacbio_bam = Path("data/pacbio.aligned.bam")
pacbio_vcf = Path("data/pacbio.vcf")
pacbio_frag = Path("data/pacbio.fragments")
pacbio_hapcut = Path("data/pacbio.hapcut")

# Helper function
def run_cmd(cmd):
    print(f"Running: {' '.join(cmd)}")
    subprocess.run(cmd, check=True)
    print("Done\n")

# Helper to count variants
def count_variants(vcf):
    """Return total, SNP, and INDEL counts for a given VCF file."""
    def _count(arg):
        res = subprocess.run(["bcftools", "view", "-v", arg, "-H", vcf],
                             capture_output=True, text=True)
        return len(res.stdout.splitlines())

    total = subprocess.run(["bcftools", "view", "-H", vcf],
                           capture_output=True, text=True)
    return {
        "Total": len(total.stdout.splitlines()),
        "SNPs": _count("snps"),
        "INDELs": _count("indels"),
    }

# ---- Illumina ----
print("Phasing Illumina sample...")

run_cmd([
    "extractHAIRS",
    "--bam", str(illumina_bam),
    "--VCF", str(illumina_vcf),
    "--out", str(illumina_frag)
])

run_cmd([
    "HAPCUT2",
    "--fragments", str(illumina_frag),
    "--VCF", str(illumina_vcf),
    "--output", str(illumina_hapcut)
])

# ---- PacBio ----
print("Phasing PacBio sample...")

run_cmd([
    "extractHAIRS",
    "--bam", str(pacbio_bam),
    "--VCF", str(pacbio_vcf),
    "--out", str(pacbio_frag)
])

run_cmd([
    "HAPCUT2",
    "--fragments", str(pacbio_frag),
    "--VCF", str(pacbio_vcf),
    "--output", str(pacbio_hapcut)
])

# output
illumina_counts = count_variants(str(illumina_vcf))
print("Illumina phasing complete:")
print("Variant summary:")
print(f"  Total variants: {illumina_counts['Total']}")
print(f"  SNPs: {illumina_counts['SNPs']}")
print(f"  INDELs: {illumina_counts['INDELs']}\n")


pacbio_counts = count_variants(str(pacbio_vcf))
print("PacBio phasing complete:")
print("Variant summary:")
print(f"  Total variants: {pacbio_counts['Total']}")
print(f"  SNPs: {pacbio_counts['SNPs']}")
print(f"  INDELs: {pacbio_counts['INDELs']}\n")


Phasing Illumina sample...
Running: extractHAIRS --bam data/illumina.aligned.bam --VCF data/illumina.vcf --out data/illumina.fragments



Extracting haplotype informative reads from bamfiles data/illumina.aligned.bam minQV 13 minMQ 20 maxIS 1000 

VCF file data/illumina.vcf has 1963 variants 
adding chrom chr10 to index 
vcffile data/illumina.vcf chromosomes 1 hetvariants 1311 variants 1963 
detected 23 variants with two non-reference alleles, these variants will not be phased
reading sorted bam/cram file data/illumina.aligned.bam 
processing reads mapped to chrom "chr10" 
final cleanup of fragment list: 38317 current chrom 0 prev 0 


[2025:11:05 13:04:12] input fragment file: data/illumina.fragments
[2025:11:05 13:04:12] input variantfile (VCF format):data/illumina.vcf
[2025:11:05 13:04:12] haplotypes will be output to file: data/illumina.hapcut
[2025:11:05 13:04:12] solution convergence cutoff: 5
[2025:11:05 13:04:12] read 1963 variants from data/illumina.vcf file 
[2025:11:05 13:04:12] read fragment file and variant file: fragments 6499 variants 1963
mean number of variants per read is 2.59 
[2025:11:05 13:04:12] bu

Done

Running: HAPCUT2 --fragments data/illumina.fragments --VCF data/illumina.vcf --output data/illumina.hapcut


[2025:11:05 13:04:12] starting to post-process phased haplotypes to further improve accuracy
[2025:11:05 13:04:12] starting to output phased haplotypes
[2025:11:05 13:04:12] OUTPUTTING PRUNED HAPLOTYPE ASSEMBLY TO FILE data/illumina.hapcut
[2025:11:05 13:04:12] N50 haplotype length is 0.41 kilobases 
[2025:11:05 13:04:12] OUTPUTTING PHASED VCF TO FILE data/illumina.hapcut.phased.VCF

Extracting haplotype informative reads from bamfiles data/pacbio.aligned.bam minQV 13 minMQ 20 maxIS 1000 

VCF file data/pacbio.vcf has 769 variants 
adding chrom chr10 to index 
vcffile data/pacbio.vcf chromosomes 1 hetvariants 354 variants 769 
detected 9 variants with two non-reference alleles, these variants will not be phased
reading sorted bam/cram file data/pacbio.aligned.bam 
processing reads mapped to chrom "chr10" 


[2025:11:05 13:04:12] input fragment file: data/pacbio.fragments
[2025:11:05 13:04:12] input variantfile (VCF format):data/pacbio.vcf
[2025:11:05 13:04:12] haplotypes will be output

Done

Phasing PacBio sample...
Running: extractHAIRS --bam data/pacbio.aligned.bam --VCF data/pacbio.vcf --out data/pacbio.fragments
Done

Running: HAPCUT2 --fragments data/pacbio.fragments --VCF data/pacbio.vcf --output data/pacbio.hapcut
Number of non-trivial connected components 21 max-Degree 235 connected variants 311 coverage-per-variant 29.556270 
Done

Illumina phasing complete:
Variant summary:
  Total variants: 1963
  SNPs: 1639
  INDELs: 324

PacBio phasing complete:
Variant summary:
  Total variants: 769
  SNPs: 695
  INDELs: 74



[2025:11:05 13:04:13] starting to post-process phased haplotypes to further improve accuracy
[2025:11:05 13:04:13] starting to output phased haplotypes
[2025:11:05 13:04:13] OUTPUTTING PRUNED HAPLOTYPE ASSEMBLY TO FILE data/pacbio.hapcut
[2025:11:05 13:04:13] N50 haplotype length is 11.88 kilobases 
[2025:11:05 13:04:13] OUTPUTTING PHASED VCF TO FILE data/pacbio.hapcut.phased.VCF


## Step 5: Compare VCFs

1) Compares Illumina and PacBio phased VCF's

2) Counts shared / unique variants

Using following ranges: 

CYP2C8 - chr10:95036772-95069497

CYP2C9 - chr10:94938658-94990091

CYP2C19 - chr10:94762681-94855547

source: https://genome.ucsc.edu/cgi-bin/hgSearch?search=CYP2C8&hgsid=3309426717_A0BBXs7VoFK7EceIdeumWtGxJn0j&db=hg38

In [86]:
from pathlib import Path
import subprocess
import pandas as pd

# --- Step 0: File paths and confirmed GRCh38 gene coordinates ---
illumina_vcf = Path("data/illumina.hapcut.phased.VCF")
pacbio_vcf   = Path("data/pacbio.hapcut.phased.VCF")

GENE_REGIONS = {
    'CYP2C19': {'chr': 'chr10', 'start': 94762681, 'end': 94855547},
    'CYP2C9':  {'chr': 'chr10', 'start': 94938658, 'end': 94990091},
    'CYP2C8':  {'chr': 'chr10', 'start': 95036772, 'end': 95069497}
}

print("Comparing phased VCFs per gene using bcftools view + filtering...")

# --- Step 1: Extract variant keys within a genomic region (no bgzip required) ---
def extract_variant_keys_in_region(vcf_path, chrom, start, end):
    """Extract variant keys (chr:pos:ref:alt) within a given region (no index required)."""
    cmd = ["bcftools", "view", "-H", str(vcf_path)]  # read entire VCF
    result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    keys = set()

    for line in result.stdout.splitlines():
        if not line.strip() or line.startswith("#"):
            continue
        fields = line.split('\t')
        v_chrom, v_pos = fields[0], int(fields[1])
        ref, alt = fields[3], fields[4]
        if v_chrom == chrom and start <= v_pos <= end:
            keys.add(f"{v_chrom}:{v_pos}:{ref}:{alt}")
    return keys

# Step 2: Compare per gene
rows = []
for gene, region in GENE_REGIONS.items():
    illumina_keys = extract_variant_keys_in_region(
        illumina_vcf, region['chr'], region['start'], region['end'])
    pacbio_keys = extract_variant_keys_in_region(
        pacbio_vcf, region['chr'], region['start'], region['end'])
    
    shared_variants = illumina_keys & pacbio_keys
    illumina_only = illumina_keys - pacbio_keys
    pacbio_only = pacbio_keys - illumina_keys
    
    rows.append({
        "Gene": gene,
        "Illumina variants": len(illumina_keys),
        "PacBio variants": len(pacbio_keys),
        "Shared": len(shared_variants),
        "Illumina-only": len(illumina_only),
        "PacBio-only": len(pacbio_only)
    })
    
    print(f"\n{gene}")
    print(f"  Illumina variants: {len(illumina_keys)}")
    print(f"  PacBio variants:   {len(pacbio_keys)}")
    print(f"  Shared variants:   {len(shared_variants)}")
    print(f"  Illumina-only:     {len(illumina_only)}")
    print(f"  PacBio-only:       {len(pacbio_only)}")

    illumina_examples = list(illumina_only)[:2]
    pacbio_examples   = list(pacbio_only)[:2]
    if illumina_examples or pacbio_examples:
        print("  Example non-shared variants:")
        for ex in illumina_examples:
            print(f"    {ex} - Illumina")
        for ex in pacbio_examples:
            print(f"    {ex} - PacBio")

# --- Step 3: Display summary table ---
summary_df = pd.DataFrame(rows)
print("\nPer-Gene Variant Comparison Summary")
display(summary_df)

print("\nComparison complete:")
print("Open discordant positions in IGV using:")
print("  - data/illumina.aligned.bam + data/pacbio.aligned.bam")
print("  - data/chr10.fa as reference")


Comparing phased VCFs per gene using bcftools view + filtering...

CYP2C19
  Illumina variants: 126
  PacBio variants:   139
  Shared variants:   109
  Illumina-only:     17
  PacBio-only:       30
  Example non-shared variants:
    chr10:94772907:G:A - Illumina
    chr10:94778981:G:A - Illumina
    chr10:94851443:Caaataaataaataaataaataaataaataaataaataaataaataaataa:Caaataaataaataaataaataaataaataaataaataaataaataa,Caaataaataaataaataaataaataaataaataaataaataa - PacBio
    chr10:94797875:C:T - PacBio

CYP2C9
  Illumina variants: 68
  PacBio variants:   72
  Shared variants:   65
  Illumina-only:     3
  PacBio-only:       7
  Example non-shared variants:
    chr10:94969284:A:C - Illumina
    chr10:94974535:gtttttttttttt:gTTtttttttttttt,gTTTTtttttttttttt - Illumina
    chr10:94941267:gtttttttttttttttttttttttttt:gTTTtttttttttttttttttttttttttt,gTTtttttttttttttttttttttttttt - PacBio
    chr10:94952992:G:A - PacBio

CYP2C8
  Illumina variants: 102
  PacBio variants:   118
  Shared variants:   98

,Gene,Illumina variants,PacBio variants,Shared,Illumina-only,PacBio-only
0,CYP2C19,126,139,109,17,30
1,CYP2C9,68,72,65,3,7
2,CYP2C8,102,118,98,4,20



Comparison complete:
Open discordant positions in IGV using:
  - data/illumina.aligned.bam + data/pacbio.aligned.bam
  - data/chr10.fa as reference


## CYP2C19 - chr10:94772907:G:A (Illumina-only)

![](CYP2C19_variant.png)

Analysis: 

- Illumina reads show a consistent G→A substitution
- PacBio shows no coverage at this locus
- Therefore, the variant appears as Illumina-only, but this may be due to absent PacBio data rather than a sequencing artifact



## CYP2C9 - chr10:94941267:gttttt... (PacBio-only)

### PacBio-only variant in a homopolymer repeat

![CYP2C9 PacBio-only variant](CYP2C9_variant.png)

Analysis: 

- The PacBio reads show insertions (purple “I” markers) and alignment breaks near a long run of T bases (ttttt…). This means PacBio is detecting extra bases or shifts within a homopolymer
- Illumina reads align smoothly, with almost all reads showing the reference sequence (no indel or alternate base)

This region is a T-homopolymer:

PacBio shows multiple small insertions (purple “I” markers) → likely false positives due to polymerase slippage vs. Illumina reads are consistent



## CYP2C8 - chr10:95043864:Gcacacacacac... (Illumina-only)

![CYP2C8 Illumina-only variant](CYP2C8_variant.png)

Analysis: 

Illumina: 

- You can see multiple purple "I" markers (insertions) stacked vertically at the same site → this means many Illumina reads report a small insertion relative to the reference
- Reads nearby show a mix of mismatches (colored bases like T, C, A) in the same region — this indicates alignment slippage


PacBio: 

- PacBio reads align smoothly through this repeat — mostly gray, minimal mismatches, and only small scattered indels
- PacBio has no consistent indel at the same position Illumina does

### Step 5: Star-Allele for each gene

## Star Allele Determination from Phased Data

Phased data allows us to determine which variants are inherited together on the same chromosome (haplotype).
Each PharmVar star-allele is defined by one or more specific variants.  
If a variant is present on a given haplotype (ex 0|1 ALT allele on the second haplotype),
we can assign the corresponding star-allele to that haplotype.

In [106]:


# PharmVar-based star-allele definitions (GRCh38 coordinates)
# only the most common, clinically relevant alleles for each gene
STAR_ALLELE_DEFINITIONS = {
    'CYP2C19': {
        '*2': {
            # rs4244285 (c.681G>A) — splicing defect → loss of function
            '94781859': ('G', 'A', 'rs4244285', 'Exon 5 splice defect, loss of function')
        },
        '*3': {
            # rs4986893 (c.636G>A) — premature stop codon → loss of function
            '94780653': ('G', 'A', 'rs4986893', 'Premature stop codon, loss of function')
        },
        '*17': {
            # rs12248560 (−806C>T in promoter) — increased transcription → gain of function
            '94761900': ('C', 'T', 'rs12248560', 'Promoter variant, increased function')
        }
    },

    'CYP2C9': {
        '*2': {
            # rs1799853 (c.430C>T, Arg144Cys) — decreased function
            '94942290': ('C', 'T', 'rs1799853', 'Arg144Cys, decreased function')
        },
        '*3': {
            # rs1057910 (c.1075A>C, Ile359Leu) — decreased function
            '94981296': ('A', 'C', 'rs1057910', 'Ile359Leu, decreased function')
        }
    },

    'CYP2C8': {
        '*3': {
            # Defined by two SNPs (rs10509681 and rs1058930) on the same haplotype
            '95038992': ('G', 'A', 'rs10509681', 'Arg139Lys, decreased activity'),
            '95067273': ('C', 'G', 'rs1058930', 'Ile269Phe, decreased activity')
        },
        '*4': {
            # rs1058930 alone — decreased activity
            '95058362': ('C', 'G', 'rs1058930', 'Ile269Phe, decreased activity')
        }
    }
}


#collect variant positions
positions = [pos for g in STAR_ALLELE_DEFINITIONS.values() for a in g.values() for pos in a.keys()]
regions = ",".join([f"10:{pos}" for pos in positions])


def get_phased_calls(vcf_file):
    # -H keeps the header lines, useful for debugging positions
    cmd = ["bcftools", "view", "-H", str(vcf_file)]
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"\nError running bcftools on {vcf_file}:")
        print(result.stderr)
        return {}
    
    calls = {}
    for line in result.stdout.strip().split("\n"):
        if not line or line.startswith("#"):
            continue
        fields = line.split("\t")
        chrom, pos = fields[0], fields[1]
        # FORMAT field is column 8; genotypes start at column 9
        fmt = fields[8].split(":")
        sample = fields[9].split(":")
        if "GT" in fmt:
            gt_index = fmt.index("GT")
            gt = sample[gt_index]
            calls[int(pos)] = gt
    return calls





illumina_vcf = "data/illumina.hapcut.phased.VCF"
pacbio_vcf   = "data/pacbio.hapcut.phased.VCF"

illumina_calls = get_phased_calls(illumina_vcf)
pacbio_calls   = get_phased_calls(pacbio_vcf)

# print("Illumina phased genotypes:", illumina_calls)
# print("PacBio   phased genotypes:", pacbio_calls)


def interpret(calls, label):
    print(f"\n===== {label} =====")
    for gene, alleles in STAR_ALLELE_DEFINITIONS.items():
        hap1, hap2 = [], []
        print(f"\n{gene}")
        for star, snps in alleles.items():
            for pos, (ref, alt, rsid, note) in snps.items():
                pos = int(pos)
                gt = calls.get(pos)
                if not gt or "|" not in gt:
                    continue
                a1, a2 = gt.split("|")
                if a1 == "1":
                    hap1.append(star)
                if a2 == "1":
                    hap2.append(star)
                print(f"  {pos} ({rsid}) {ref}>{alt} GT={gt} → "
                      f"{'hap1:'+star if a1=='1' else ''} {'hap2:'+star if a2=='1' else ''}")
        hap1 = hap1[0] if hap1 else "*1"
        hap2 = hap2[0] if hap2 else "*1"
        print(f"→ Diplotype: {gene} {hap1}/{hap2}")

interpret(illumina_calls, "ILLUMINA")
interpret(pacbio_calls,   "PACBIO")


# --- utilities ---------------------------------------------------------------

def gene_positions(star_defs):
    # STAR_ALLELE_DEFINITIONS: {gene: { '*1': {pos: {...}}, '*17': {pos: {...}}, ...}}
    out = {}
    for gene, alleles in STAR_ALLELE_DEFINITIONS.items():
        posset = set()
        for allele, posmap in alleles.items():
            posset.update(posmap.keys())
        out[gene] = sorted(posset)
    return out

def hap_counts(calls, positions_by_gene):
    # calls: {pos:int -> '0|1','1|0','1/1','0/1','0/0','./.'}
    counts = {}
    for gene, poslist in positions_by_gene.items():
        h1 = h2 = unph = 0
        for p in poslist:
            gt = calls.get(p)
            if not gt: 
                continue
            if '|' in gt:                    # phased
                a, b = gt.split('|')
                if a == '1': h1 += 1
                if b == '1': h2 += 1
            elif '/' in gt:                  # unphased
                a, b = gt.split('/')
                if a == '1' or b == '1': 
                    unph += 1
        counts[gene] = {'hap1': h1, 'hap2': h2, 'unphased': unph}
    return counts

def call_star_simple(calls, star_defs):
    """
    Very lightweight star calling using genotypes only.
    If all defining positions for a star allele are present with ALT on a haplotype,
    we assign that haplotype that star; otherwise '*1'.
    NOTE: This ignores specific ALT bases and compound rules; it's a practical
    check for your assignment figures. For strict calls, also verify ALT base.
    """
    per_gene = {}
    for gene, alleles in star_defs.items():
        h1_star, h2_star = '*1', '*1'
        # Try non-*1 alleles first so they override if matched
        for star, posmap in alleles.items():
            if star == '*1':
                continue
            # Require every defining position to have ALT on a given hap
            h1_ok = h2_ok = True
            any_seen = False
            for pos in posmap.keys():
                gt = calls.get(pos)
                if not gt or gt in ('./.',):
                    h1_ok = h2_ok = False
                    continue
                any_seen = True
                if '|' in gt:
                    a,b = gt.split('|')
                    if a != '1': h1_ok = False
                    if b != '1': h2_ok = False
                elif '/' in gt:
                    # unphased ALT can't be confidently assigned; mark both false
                    a,b = gt.split('/')
                    if not (a=='1' or b=='1'):
                        h1_ok = h2_ok = False
                    else:
                        h1_ok = h2_ok = False  # keep conservative for hap assignment
                else:
                    h1_ok = h2_ok = False
            if any_seen and h1_ok: h1_star = star
            if any_seen and h2_ok: h2_star = star
        per_gene[gene] = (h1_star, h2_star)
    return per_gene

def print_report(title, calls, positions_by_gene, star_defs):
    print(title)
    counts = hap_counts(calls, positions_by_gene)
    stars  = call_star_simple(calls, star_defs)
    for gene in sorted(positions_by_gene.keys()):
        c = counts[gene]
        h1, h2 = stars[gene]
        print(f"GENE: {gene}\n")
        print("Illumina Haplotypes:" if 'Illumina' in title else "PacBio Haplotypes:")
        print(f"  Haplotype 1: {c['hap1']} variants")
        print(f"  Haplotype 2: {c['hap2']} variants")
        if c['unphased']>0:
            print(f"  (Unphased in sample: {c['unphased']})")
        print(f"  Haplotype 1: {h1} (reference/wild-type)" if h1=='*1' else f"  Haplotype 1: {h1}")
        print(f"  Haplotype 2: {h2} (reference/wild-type)" if h2=='*1' else f"  Haplotype 2: {h2}")
        print("\n  CONSENSUS DIPLOTYPE:")
        print(f"    {gene} {h1}/{h2}\n")

# --- run it ------------------------------------------------------------------

positions_by_gene = gene_positions(STAR_ALLELE_DEFINITIONS)

print_report("===== ILLUMINA =====", illumina_calls, positions_by_gene, STAR_ALLELE_DEFINITIONS)
print()
print_report("===== PACBIO =====",   pacbio_calls,   positions_by_gene, STAR_ALLELE_DEFINITIONS)




===== ILLUMINA =====

CYP2C19
→ Diplotype: CYP2C19 *1/*1

CYP2C9
→ Diplotype: CYP2C9 *1/*1

CYP2C8
→ Diplotype: CYP2C8 *1/*1

===== PACBIO =====

CYP2C19
  94761900 (rs12248560) C>T GT=0|1 →  hap2:*17
→ Diplotype: CYP2C19 *1/*17

CYP2C9
→ Diplotype: CYP2C9 *1/*1

CYP2C8
→ Diplotype: CYP2C8 *1/*1
===== ILLUMINA =====
GENE: CYP2C19

PacBio Haplotypes:
  Haplotype 1: 0 variants
  Haplotype 2: 0 variants
  Haplotype 1: *1 (reference/wild-type)
  Haplotype 2: *1 (reference/wild-type)

  CONSENSUS DIPLOTYPE:
    CYP2C19 *1/*1

GENE: CYP2C8

PacBio Haplotypes:
  Haplotype 1: 0 variants
  Haplotype 2: 0 variants
  Haplotype 1: *1 (reference/wild-type)
  Haplotype 2: *1 (reference/wild-type)

  CONSENSUS DIPLOTYPE:
    CYP2C8 *1/*1

GENE: CYP2C9

PacBio Haplotypes:
  Haplotype 1: 0 variants
  Haplotype 2: 0 variants
  Haplotype 1: *1 (reference/wild-type)
  Haplotype 2: *1 (reference/wild-type)

  CONSENSUS DIPLOTYPE:
    CYP2C9 *1/*1


===== PACBIO =====
GENE: CYP2C19

PacBio Haplotypes:
  Ha

Step Notes: The PacBio results identified a CYP2C19*17 gain-of-function allele (rs12248560) on the second haplotype, while the Illumina data showed only the reference *1/*1 genotype. All other genes (CYP2C9 and CYP2C8) were consistent with wild-type (*1/*1) across both technologies. This suggests that the PacBio long-read sequencing captured a phased promoter variant that short-read Illumina data likely missed due to alignment or coverage limitations.